# ml-project-iscb-jovian-mutation-classify

Use the "Run" button to execute the code.

In [1]:
#!pip install jovian --upgrade --quiet

In [27]:
pip install xgboost --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.


distutils: C:\Users\anubr\anaconda3\Include\UNKNOWN
sysconfig: C:\Users\anubr\anaconda3\Include
user = False
home = None
root = None
prefix = None
distutils: C:\Users\anubr\anaconda3\Include\UNKNOWN
sysconfig: C:\Users\anubr\anaconda3\Include
user = False
home = None
root = None
prefix = None


In [28]:
import jovian

In [29]:
# Execute this to save new versions of the notebook
jovian.commit(project="ml-project-iscb-jovian-mutation-classify")

<IPython.core.display.Javascript object>

[jovian] Updating notebook "anubratadas/ml-project-iscb-jovian-mutation-classify" on https://jovian.ai/
[jovian] Committed successfully! https://jovian.ai/anubratadas/ml-project-iscb-jovian-mutation-classify


'https://jovian.ai/anubratadas/ml-project-iscb-jovian-mutation-classify'

In [30]:
import pandas as pd
import numpy as np
#import requests
#import xml.etree.ElementTree as ET
import time
import re
from tqdm import tqdm
%matplotlib inline

In [31]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.simplefilter(action='ignore')

In [32]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
# sns.set_style('darkgrid')
# matplotlib.rcParams['font.size'] = 14
#matplotlib.rcParams['figure.figsize'] = (9, 5)
# matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [33]:
tqdm.pandas(desc="progress bar")

In [34]:
from collections import Counter

In [35]:
# utilities
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import class_weight

# numerical transformation of text
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# machine learning classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# scoring matrices
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

#### train data

In [36]:
train_text_df = pd.read_csv('training_text.zip', sep="\|\|", encoding="utf-8",engine="python", skiprows=1, names=["ID", "Text"],compression='zip')

In [37]:
train_variant_df = pd.read_csv('training_variants.zip',compression='zip')

In [38]:
train_text_df.shape, train_variant_df.shape

((3321, 2), (3321, 4))

In [39]:
train_text_df.columns, train_variant_df.columns

(Index(['ID', 'Text'], dtype='object'),
 Index(['ID', 'Gene', 'Variation', 'Class'], dtype='object'))

In [40]:
# merging the text and variation data
train_df = pd.merge(train_variant_df,train_text_df,on='ID')

In [54]:
train_text_df.head(n=4)

,ID,Text
0,0,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,Abstract Background Non-small cell lung canc...
2,2,Abstract Background Non-small cell lung canc...
3,3,Recent evidence has demonstrated that acquired...


In [42]:
train_variant_df.head()

,ID,Gene,Variation,Class
0,0,FAM58A,Truncating Mutations,1
1,1,CBL,W802*,2
2,2,CBL,Q249E,2
3,3,CBL,N454D,3
4,4,CBL,L399V,4


In [55]:
train_df.head()

,ID,Gene,Variation,Class,Text,var_in_txt,any_vr_txt,comn_mutns
0,0,FAM58A,Truncating Mutations,1,Cyclin-dependent kinases (CDKs) regulate a var...,0,0,22
1,1,CBL,W802*,2,Abstract Background Non-small cell lung canc...,1,9,16
2,2,CBL,Q249E,2,Abstract Background Non-small cell lung canc...,1,9,16
3,3,CBL,N454D,3,Recent evidence has demonstrated that acquired...,1,7,12
4,4,CBL,L399V,4,Oncogenic mutations in the monomeric Casitas B...,1,15,37


In [44]:
# label all row indices where the variation is found in the text as 1 else label row as 0
def var_in_txt(df):
    idx_list=[]
    for row in df.iterrows():
        if (isinstance(row[1][4],str) and row[1][2] in  row[1][4]):
            idx_list.append(1)
        elif (isinstance(row[1][4],str) and row[1][2] not in row[1][4]):
            idx_list.append(0)
        else:    
            idx_list.append(-1)
    assert (len(idx_list) == df.shape[0]),"all rows not covered"        
    return idx_list        

In [45]:
idx_list = var_in_txt(train_df)

In [46]:
train_df['var_in_txt']=idx_list

In [47]:
trn_var_lst = train_df.Variation.tolist()

In [48]:
# label all row indices where any of the variations is found in the text as 1 else label row as 0
def any_var_in_txt(txt,lst):   
    var_lst = [item for item in lst if str(item) in str(txt)]
    if any(var_lst):
        return (len(var_lst))
    else:
        return (0)    

In [49]:
train_df['any_vr_txt'] = train_df.Text.progress_apply(any_var_in_txt,args=(trn_var_lst,))

progress bar: 100%|██████████| 3321/3321 [12:56<00:00,  4.28it/s]


In [50]:
def common_mutations_len(text):
    if isinstance(text,str):
        match_pat = re.compile('([A-Z][\d]+[A-Z])')
        xval=Counter(re.findall(match_pat,text))
        return len(xval)
    return 0

In [51]:
train_df["comn_mutns"]=train_df['Text'].progress_map(common_mutations_len)

progress bar: 100%|██████████| 3321/3321 [00:05<00:00, 615.28it/s]


In [56]:
train_df.head()

,ID,Gene,Variation,Class,Text,var_in_txt,any_vr_txt,comn_mutns
0,0,FAM58A,Truncating Mutations,1,Cyclin-dependent kinases (CDKs) regulate a var...,0,0,22
1,1,CBL,W802*,2,Abstract Background Non-small cell lung canc...,1,9,16
2,2,CBL,Q249E,2,Abstract Background Non-small cell lung canc...,1,9,16
3,3,CBL,N454D,3,Recent evidence has demonstrated that acquired...,1,7,12
4,4,CBL,L399V,4,Oncogenic mutations in the monomeric Casitas B...,1,15,37


In [57]:
numerics_df = train_df[['var_in_txt', 'any_vr_txt','comn_mutns']]

In [58]:
numerics_df.isna().sum()

var_in_txt    0
any_vr_txt    0
comn_mutns    0
dtype: int64

In [59]:
y_train = train_df['Class']

In [60]:
numerics_df.shape, y_train.shape

((3321, 3), (3321,))

#### test data

In [65]:
test_text_df = pd.read_csv('stage2_test_text.csv', sep="\|\|", encoding="utf-8",engine="python", skiprows=1, names=["ID", "Text"])

In [66]:
test_variant_df = pd.read_csv('stage2_test_variants.csv')

In [67]:
# merging the text and variation data
test_df = pd.merge(test_variant_df,test_text_df,on='ID')

In [68]:
test_df.head()

,ID,Gene,Variation,Text
0,1,CHEK2,H371Y,The incidence of breast cancer is increasing i...
1,2,AXIN2,Truncating Mutations,An unselected series of 310 colorectal carcino...
2,3,WNT4,E216G,Mycosis fungoides and Sézary syndrome are prim...
3,4,SUCLA2,G118R,Regulated progression through the cell cycle ...
4,5,BRAF,T599insTT,Pilocytic astrocytoma (PA) is emerging as a tu...


In [69]:
# label all row indices where the variation is found in the text as 1 else label row as 0
def var_in_txt(df):
    idx_list=[]
    for row in tqdm(df.iterrows()):
        if (isinstance(row[1][3],str) and row[1][2] in  row[1][3]):
            idx_list.append(1)
        elif (isinstance(row[1][3],str) and row[1][2] not in row[1][3]):
            idx_list.append(0)
        else:    
            idx_list.append(-1)
    assert (len(idx_list) == df.shape[0]),"all rows not covered"        
    return idx_list        

In [70]:
test_idx_lst = var_in_txt(test_df)

986it [00:00, 4131.92it/s]


In [71]:
test_df['var_in_txt'] = test_idx_lst

In [72]:
test_var_lst = test_df.Variation.tolist()

In [73]:
test_df['any_vr_txt'] = test_df.Text.progress_apply(any_var_in_txt,args=(test_var_lst,))

progress bar: 100%|██████████| 986/986 [01:06<00:00, 14.79it/s]


In [74]:
test_df["comn_mutns"] = test_df['Text'].progress_map(common_mutations_len)

progress bar: 100%|██████████| 986/986 [00:01<00:00, 885.77it/s]


In [77]:
test_numerics_df = test_df[['var_in_txt', 'any_vr_txt','comn_mutns']]

In [78]:
test_numerics_df.isna().sum()

var_in_txt    0
any_vr_txt    0
comn_mutns    0
dtype: int64

###  ML models

#### Logistic regression

In [89]:
columns = ['ID','class1','class2','class3','class4','class5','class6','class7','class8','class9']

In [82]:
def grid_search_log_reg(x_train,y_train,parameters):
    model = LogisticRegression(multi_class='multinomial') # define the multinomial logistic regression model         
    clf = GridSearchCV(model, parameters,cv=5,scoring='neg_log_loss',verbose =0)
    clf.fit(x_train,y_train)
    bst_estm = clf.best_estimator_
    bst_params = clf.best_params_   
    bst_score = clf.best_score_
    predict_proba_train = clf.predict_proba(x_train) 
    result = {"predict proba train":predict_proba_train,"best score":bst_score,"best estimator":bst_estm,"best params":bst_params}
    return result,clf

In [66]:
LR_parameters = {'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],'penalty':['l1', 'l2', 'elasticnet', 'none'],'C':[0.0001,0.01,0.1,1.0,10.0]}

In [83]:
LR_numerics_result, LR_model = grid_search_log_reg(numerics_df,y_train,LR_parameters)

In [108]:
numerics_LR_logloss = log_loss(y_train,LR_numerics_result['predict proba train'])

In [109]:
numerics_LR_logloss

1.7748878489682127

In [93]:
print("best score:negative log loss ",LR_numerics_result['best score'])

best score:negative log loss  -1.805753453400375


In [94]:
print("best estimator ",LR_numerics_result['best estimator'])

best estimator  LogisticRegression(C=0.01, multi_class='multinomial', solver='newton-cg')


In [95]:
print("best parameters ",LR_numerics_result['best params'])

best parameters  {'C': 0.01, 'penalty': 'l2', 'solver': 'newton-cg'}


stage2 test data

In [127]:
stg2_LR_y_numerics = LR_model.predict_proba(test_numerics_df)
stg2_LR_y_numerics = pd.DataFrame(stg2_LR_y_numerics).reset_index() # getting index as column
stg2_LR_y_numerics['index'] = stg2_LR_y_numerics['index']+1  # starting index from 1
stg2_LR_y_numerics.columns = columns

In [128]:
stg2_LR_y_numerics.head()

,ID,class1,class2,class3,class4,class5,class6,class7,class8,class9
0,1,0.157021,0.149138,0.027910,0.227148,0.062842,0.059726,0.296065,0.006113,0.014036
1,2,0.207729,0.176275,0.027526,0.166416,0.060711,0.060314,0.280891,0.006310,0.013829
2,3,0.154563,0.120908,0.035164,0.252154,0.080510,0.077885,0.252891,0.006057,0.019868
3,4,0.156990,0.155010,0.026561,0.221709,0.059604,0.056448,0.304525,0.006104,0.013050
4,5,0.206236,0.191598,0.024355,0.156206,0.053300,0.052515,0.297957,0.006246,0.011588


In [98]:
stg2_LR_y_numerics.to_csv("stg2_LR_y_numerics.csv",index=False,header=True)

#### Naive Bayes

In [102]:
# Naive Bayes doesn't work with negative values, hence replace -1 with 0
numerics_df['var_in_txt'] = np.where(numerics_df.var_in_txt<0,0,numerics_df.var_in_txt)

In [48]:
def grid_search_naive_bayes(x_train,y_train,parameters):
    model = MultinomialNB() # define the Naive Bayes model        
    clf = GridSearchCV(model, parameters,cv=5,scoring='neg_log_loss',verbose =0)
    clf.fit(x_train,y_train)
    bst_estm = clf.best_estimator_
    bst_params = clf.best_params_   
    bst_score = clf.best_score_
    predict_proba_train = clf.predict_proba(x_train) 
    result = {"predict proba train":predict_proba_train,"best score":bst_score,"best estimator":bst_estm,"best params":bst_params}
    return result,clf

In [121]:
NB_parameters ={"alpha":[1.0,10.0,100.0,400.0,500.0,800.0,1000.0]}

In [122]:
NB_numerics_result, NB_model = grid_search_naive_bayes(numerics_df,y_train,NB_parameters)

In [123]:
numerics_NB_logloss = log_loss(y_train,NB_numerics_result['predict proba train'])

In [124]:
numerics_NB_logloss

3.7050208523616495

In [125]:
print("best score:negative log loss ",NB_numerics_result['best score'])

best score:negative log loss  -3.9038126231011936


In [126]:
print("best estimator ",NB_numerics_result['best estimator'])

best estimator  MultinomialNB(alpha=400.0)


stage 2 test data

In [130]:
stg2_NB_y_numerics = NB_model.predict_proba(test_numerics_df)
stg2_NB_y_numerics = pd.DataFrame(stg2_NB_y_numerics).reset_index() # getting index as column
stg2_NB_y_numerics['index'] = stg2_NB_y_numerics['index']+1  # starting index from 1
stg2_NB_y_numerics.columns = columns

In [131]:
stg2_NB_y_numerics.head()

,ID,class1,class2,class3,class4,class5,class6,class7,class8,class9
0,1,0.039145,0.012423,0.161822,0.205178,0.186222,0.131734,0.014229,0.004137,0.245111
1,2,0.075061,0.021142,0.079703,0.332510,0.202911,0.182762,0.037298,0.001239,0.067374
2,3,0.001834,0.000123,0.133771,0.067537,0.139464,0.069618,0.000107,0.000067,0.587479
3,4,0.064015,0.027743,0.149050,0.227199,0.174946,0.132694,0.033521,0.008361,0.182472
4,5,0.153680,0.089555,0.042753,0.274106,0.113976,0.120795,0.178986,0.004159,0.021991


In [132]:
stg2_NB_y_numerics.to_csv("stg2_NB_y_numerics.csv",index=False,header=True)

#### Decision Tree Classifier

In [79]:
DTC_params = {'criterion':["gini","entropy"],"splitter":['best','random'],
             "max_depth":[2,6,10,20,100],"min_samples_split":[1,2,5,10,20,50,100],
             "min_samples_leaf":[1,5,10,100,200],"min_weight_fraction_leaf":[0.0,0.01,0.1,0.3],
             "max_features":["auto","sqrt","log2"],"class_weight":["none","balanced"],
             "ccp_alpha":[0.0,0.01,1.0,10.0]}

In [80]:
def grid_search_decision_tree(x_train,y_train,parameters):
    model = DecisionTreeClassifier(random_state = 42) # define the Decision Tree model          
    clf = GridSearchCV(model, parameters,cv=5,scoring='neg_log_loss',verbose = 0)
    clf.fit(x_train,y_train)
    bst_estm = clf.best_estimator_
    bst_params = clf.best_params_   
    bst_score = clf.best_score_
    predict_proba_train = clf.predict_proba(x_train) 
    result = {"predict proba train":predict_proba_train,"best score":bst_score,"best estimator":bst_estm,"best params":bst_params}
    return result,clf

In [81]:
DTC_numerics_result, DTC_model = grid_search_decision_tree(numerics_df,y_train,DTC_params)

In [82]:
numerics_DTC_logloss = log_loss(y_train,DTC_numerics_result['predict proba train'])

In [83]:
numerics_DTC_logloss

2.173102637761483

In [84]:
print("best score:negative log loss ",DTC_numerics_result['best score'])

best score:negative log loss  -2.190482489694527


In [85]:
print("best estimator ",DTC_numerics_result['best estimator'])

best estimator  DecisionTreeClassifier(ccp_alpha=0.01, class_weight='balanced', max_depth=20,
                       max_features='auto', min_samples_split=5,
                       random_state=42)


In [86]:
print("best parameters ",DTC_numerics_result['best params'])

best parameters  {'ccp_alpha': 0.01, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 20, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0.0, 'splitter': 'best'}


#### Random Forest Model

In [89]:
RF_params = {'bootstrap': [True], 'max_depth': [5, 10,20,None], 'max_features': ['auto', 'log2','sqrt'], 
              'n_estimators': [5,7, 20, 50, 75, 100, 150],'min_samples_split':[2,4,8],'min_samples_leaf':[1,5,20,100]}

In [90]:
def grid_search_randomforest_model(x_train,y_train,parameters):
    model = RandomForestClassifier(random_state = 42,verbose=0) # define the Decision Tree model          
    clf = GridSearchCV(model, parameters,cv=5,scoring='neg_log_loss',verbose = 0)
    clf.fit(x_train,y_train)
    bst_estm = clf.best_estimator_
    bst_params = clf.best_params_   
    bst_score = clf.best_score_
    predict_proba_train = clf.predict_proba(x_train) 
    result = {"predict proba train":predict_proba_train,"best score":bst_score,"best estimator":bst_estm,"best params":bst_params}
    return result,clf

In [92]:
RF_numerics_result, RF_model = grid_search_randomforest_model(numerics_df,y_train,RF_params)

In [97]:
numerics_RF_logloss = log_loss(y_train,RF_numerics_result['predict proba train'])

In [98]:
numerics_RF_logloss

1.2930524211230363

In [99]:
print("best score:negative log loss ",RF_numerics_result['best score'])

best score:negative log loss  -1.7432693827021861


In [100]:
print("best estimator ",RF_numerics_result['best estimator'])

best estimator  RandomForestClassifier(max_depth=10, min_samples_leaf=5, n_estimators=150,
                       random_state=42)


In [101]:
print("best parameters ",RF_numerics_result['best params'])

best parameters  {'bootstrap': True, 'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 5, 'min_samples_split': 2, 'n_estimators': 150}


#### XGBoost model

In [50]:
def xgboost_model(x_train,y_train):    
    model = XGBClassifier(n_estimators = 200, objective="multi:softproba",eval_metric="logloss",
                          verbosity=0,tree_method="approx",seed=42) # define the XGBoost model
    model.fit(x_train,y_train)   
    predict_proba_train = model.predict_proba(x_train)    
    predict_train = model.predict(x_train)        
    result = {"predict_proba_train":predict_proba_train,"predict_train":predict_train}           
    return result  

In [51]:
new_y_train = y_train-1

In [52]:
numerics_result = xgboost_model(numerics_df,new_y_train)

In [53]:
numerics_XGB_logloss = log_loss(y_train,numerics_result['predict_proba_train'])

In [54]:
numerics_XGB_logloss

0.703006190040045

#### Test Data

var_in_txt    0
any_vr_txt    0
comn_mutns    0
dtype: int64